In [11]:
import requests
import time
import base64
import hashlib
import hmac
import json
import urllib.request as urllib2
import ssl
import pandas as pd
import ast
import datetime
import urllib.parse
import hashlib
import hmac
import base64

class API(object):
    
    def __init__(self, apiPath, apiPublicKey="", apiPrivateKey="", timeout=10, checkCertificate=True, useNonce=False, apiversion='0'):
        self.apiPath = apiPath
        self.apiPublicKey = apiPublicKey
        self.apiPrivateKey = apiPrivateKey
        self.timeout = timeout
        self.nonce = 0
        self.apiversion = apiversion
        self.session = requests.Session()
        self.checkCertificate = checkCertificate
        self.useNonce = useNonce
        self.response = None
        self._json_options = {}
        
    def get_apiversion(self):
        return self.apiversion
    
    def json_options(self, **kwargs):
        """ Set keyword arguments to be passed to JSON deserialization.
        :param kwargs: passed to :py:meth:`requests.Response.json`
        :returns: this instance for chaining
        """
        self._json_options = kwargs
        return self

    def close(self):
        """ Close this session.
        :returns: None
        """
        self.session.close()
        return

    def load_key(self, path):
        """ Load key and secret from file.
        Expected file format is key and secret on separate lines.
        :param path: path to keyfile
        :type path: str
        :returns: None
        """
        with open(path, 'r') as f:
            self.apiPublicKey = f.readline().strip()
            self.apiPrivateKey = f.readline().strip()
        return

    def _query(self, urlpath, data, headers=None, timeout=None):
        """ Low-level query handling.
        .. note::
           Use :py:meth:`query_private` or :py:meth:`query_public`
           unless you have a good reason not to.
        :param urlpath: API URL path sans host
        :type urlpath: str
        :param data: API request parameters
        :type data: dict
        :param headers: (optional) HTTPS headers
        :type headers: dict
        :param timeout: (optional) if not ``None``, a :py:exc:`requests.HTTPError`
                        will be thrown after ``timeout`` seconds if a response
                        has not been received
        :type timeout: int or float
        :returns: :py:meth:`requests.Response.json`-deserialised Python object
        :raises: :py:exc:`requests.HTTPError`: if response status not successful
        """
        if data is None:
            data = {}
        if headers is None:
            headers = {}

        url = self.apiPath + urlpath

        self.response = self.session.post(url, data = data, headers = headers,
                                          timeout = timeout)

        return self.response.json(**self._json_options)
    
#%%
class API_spot(API):   
    def __init__(self, apiPublicKey="", apiPrivateKey="", timeout=10, checkCertificate=True, useNonce=False):
        super().__init__('https://api.kraken.com', apiPublicKey, apiPrivateKey, timeout, checkCertificate, useNonce,'0')
    
    def query_public(self, method, data=None, timeout=None):
        """ Performs an API query that does not require a valid key/secret pair.
        :param method: API method name
        :type method: str
        :param data: (optional) API request parameters
        :type data: dict
        :param timeout: (optional) if not ``None``, a :py:exc:`requests.HTTPError`
                        will be thrown after ``timeout`` seconds if a response
                        has not been received
        :type timeout: int or float
        :returns: :py:meth:`requests.Response.json`-deserialised Python object
        """
        if data is None:
            data = {}

        urlpath = '/' + self.apiversion + '/public/' + method
        
        return self._query(urlpath, data, timeout = timeout)
    
    def query_private(self, method, data=None, timeout=None):
        """ Performs an API query that requires a valid key/secret pair.
        :param method: API method name
        :type method: str
        :param data: (optional) API request parameters
        :type data: dict
        :param timeout: (optional) if not ``None``, a :py:exc:`requests.HTTPError`
                        will be thrown after ``timeout`` seconds if a response
                        has not been received
        :type timeout: int or float
        :returns: :py:meth:`requests.Response.json`-deserialised Python object
        """
        if data is None:
            data = {}

        if not self.apiPublicKey or not self.apiPrivateKey:
            raise Exception('Either key or secret is not set! (Use `load_key()`.')

        data['nonce'] = self._nonce()

        urlpath = '/' + self.apiversion + '/private/' + method

        headers = {
            'API-Key': self.apiPublicKey,
            'API-Sign': self._sign(data, urlpath)
        }

        return self._query(urlpath, data, headers, timeout = timeout)
    
    def _nonce(self):
        """ Nonce counter.
        :returns: an always-increasing unsigned integer (up to 64 bits wide)
        """
        return int(1000*time.time())

    def _sign(self, data, urlpath):
        """ Sign request data according to Kraken's scheme.
        :param data: API request parameters
        :type data: dict
        :param urlpath: API URL path sans host
        :type urlpath: str
        :returns: signature digest
        """
        postdata = urllib.parse.urlencode(data)

        # Unicode-objects must be encoded before hashing
        encoded = (str(data['nonce']) + postdata).encode()
        message = urlpath.encode() + hashlib.sha256(encoded).digest()

        signature = hmac.new(base64.b64decode(self.apiPrivateKey),
                             message, hashlib.sha512)
        sigdigest = base64.b64encode(signature.digest())

        return sigdigest.decode()
    
    def get_bid(self,ticker):
        tickerinfo = self.query_public("Ticker", {"pair":ticker})
        bid = tickerinfo['result'][ticker]['b'][0]
        return float(bid)
    
    def get_ask(self,ticker):
        tickerinfo = self.query_public("Ticker", {"pair":ticker})
        ask = tickerinfo['result'][ticker]['a'][0]
        return float(ask)
    
    def get_mid(self,ticker):
        bid = self.get_bid(ticker)
        ask = self.get_ask(ticker)
        mid=(bid+ask)/2
        return mid
    
    def get_balance(self,ccy):
        return self.query_private('Balance')['result'][ccy]
    
    def create_order(self,ticker,side,orderType,price,volume):
        response = self.query_private('AddOrder',
                       {'pair':ticker,
                        'type':side,
                        'ordertype':orderType,
                        'price':price,
                        'volume':volume,
                        })
        return response
    
    def transfer_from_spot_to_future(self, ccy, amount):
        response = self.query_private('WalletTransfer', {'asset':ccy,
                                          'from':'Spot Wallet',
                                          'to':'Futures Wallet',
                                          'amount':amount})
        return response

class API_future(API):
    def __init__(self, apiPublicKey="", apiPrivateKey="", timeout=10, checkCertificate=True, useNonce=False):
        super().__init__('https://futures.kraken.com/derivatives', apiPublicKey, apiPrivateKey, timeout, checkCertificate, useNonce,'api/v3')
        self.apiPath ="https://www.cryptofacilities.com/derivatives"
    
    def query_public(self, method, data=None, timeout=None):
        """ Performs an API query that does not require a valid key/secret pair.
        :param method: API method name
        :type method: str
        :param data: (optional) API request parameters
        :type data: dict
        :param timeout: (optional) if not ``None``, a :py:exc:`requests.HTTPError`
                        will be thrown after ``timeout`` seconds if a response
                        has not been received
        :type timeout: int or float
        :returns: :py:meth:`requests.Response.json`-deserialised Python object
        """
        if data is None:
            data = {}

        urlpath = '/' + self.apiversion + "/"+ method + "?" + "symbol=%s" % data["pair"]
        #return urlpath
        return self._query(urlpath, data, timeout = timeout)
    
    def query_private(self, method, data=None, timeout=None):
        """ Performs an API query that requires a valid key/secret pair.
        :param method: API method name
        :type method: str
        :param data: (optional) API request parameters
        :type data: dict
        :param timeout: (optional) if not ``None``, a :py:exc:`requests.HTTPError`
                        will be thrown after ``timeout`` seconds if a response
                        has not been received
        :type timeout: int or float
        :returns: :py:meth:`requests.Response.json`-deserialised Python object
        """
        if data is None:
            data = {}

        if not self.apiPublicKey or not self.apiPrivateKey:
            raise Exception('Either key or secret is not set! (Use `load_key()`.')

        data['nonce'] = self._nonce()

        urlpath = '/' + self.apiversion + '/' + method

        headers = {
            'API-Key': self.apiPublicKey,
            'API-Sign': self.sign_message(urlpath)
        }

        return self._query(urlpath, data, headers, timeout = timeout)
    
    def get_instruments(self):
        endpoint = "/api/v3/instruments"
        return self.make_request("GET", endpoint)

    # returns market data for all instruments
    def get_tickers(self):
        endpoint = "/api/v3/tickers"
        return self.make_request("GET", endpoint)

    # returns the entire order book of a futures
    def get_orderbook(self, symbol):
        endpoint = "/api/v3/orderbook"
        postUrl = "symbol=%s" % symbol
        return self.make_request("GET", endpoint, postUrl=postUrl)

    # returns historical data for futures and indices
    def get_history(self, symbol, lastTime=""):
        endpoint = "/api/v3/history"
        if lastTime != "":
            postUrl = "symbol=%s&lastTime=%s" % (symbol, lastTime)
        else:
            postUrl = "symbol=%s" % symbol
        return self.make_request("GET", endpoint, postUrl=postUrl)

    ##### private endpoints #####

    # returns key account information
    # Deprecated because it returns info about the Futures margin account
    # Use get_accounts instead
    def get_account(self):
        endpoint = "/api/v3/account"
        return self.make_request("GET", endpoint)

    # returns key account information
    def get_accounts(self):
        endpoint = "/api/v3/accounts"
        return self.make_request("GET", endpoint)

    # places an order
    def send_order(self, orderType, symbol, side, size, limitPrice, stopPrice=None, clientOrderId=None):
        endpoint = "/api/v3/sendorder"
        postBody = "orderType=%s&symbol=%s&side=%s&size=%s&limitPrice=%s" % (orderType, symbol, side, size, limitPrice)

        if orderType == "stp" and stopPrice is not None:
            postBody += "&stopPrice=%s" % stopPrice

        if clientOrderId is not None:
            postBody += "&cliOrdId=%s" % clientOrderId

        return self.make_request("POST", endpoint, postBody=postBody)

    # places an order
    def send_order_1(self, order):
        endpoint = "/api/v3/sendorder"
        postBody = urllib.parse.urlencode(order)
        return self.make_request("POST", endpoint, postBody=postBody)

    # edit an order
    def edit_order(self, edit):
        endpoint = "/api/v3/editorder"
        postBody = urllib.parse.urlencode(edit)
        return self.make_request("POST", endpoint, postBody=postBody)

    # cancels an order
    def cancel_order(self, order_id=None, cli_ord_id=None):
        endpoint = "/api/v3/cancelorder"

        if order_id is None:
            postBody = "cliOrdId=%s" % cli_ord_id
        else:
            postBody = "order_id=%s" % order_id

        return self.make_request("POST", endpoint, postBody=postBody)

    # cancel all orders
    def cancel_all_orders(selfs, symbol=None):
        endpoint = "/api/v3/cancelallorders"
        if symbol is not None:
            postbody = "symbol=%s" % symbol
        else:
            postbody = ""

        return selfs.make_request("POST", endpoint, postBody=postbody)

    # cancel all orders after
    def cancel_all_orders_after(selfs, timeoutInSeconds=60):
        endpoint = "/api/v3/cancelallordersafter"
        postbody = "timeout=%s" % timeoutInSeconds

        return selfs.make_request("POST", endpoint, postBody=postbody)

    # places or cancels orders in batch
    def send_batchorder(self, jsonElement):
        endpoint = "/api/v3/batchorder"
        postBody = "json=%s" % jsonElement
        return self.make_request("POST", endpoint, postBody=postBody)

    # returns all open orders
    def get_openorders(self):
        endpoint = "/api/v3/openorders"
        return self.make_request("GET", endpoint)

    # returns filled orders
    def get_fills(self, lastFillTime=""):
        endpoint = "/api/v3/fills"
        if lastFillTime != "":
            postUrl = "lastFillTime=%s" % lastFillTime
        else:
            postUrl = ""
        return self.make_request("GET", endpoint, postUrl=postUrl)

    # returns all open positions
    def get_openpositions(self):
        endpoint = "/api/v3/openpositions"
        return self.make_request("GET", endpoint)

    # return the user recent orders
    def get_recentorders(self, symbol=""):
        endpoint = "/api/v3/recentorders"
        if symbol != "":
            postUrl = "symbol=%s" % symbol
        else:
            postUrl = ""
        return self.make_request("GET", endpoint, postUrl=postUrl)

    # sends an xbt withdrawal request
    def send_withdrawal(self, targetAddress, currency, amount):
        endpoint = "/api/v3/withdrawal"
        postBody = "targetAddress=%s&currency=%s&amount=%s" % (targetAddress, currency, amount)
        return self.make_request("POST", endpoint, postBody=postBody)

    # returns xbt transfers
    def get_transfers(self, lastTransferTime=""):
        endpoint = "/api/v3/transfers"
        if lastTransferTime != "":
            postUrl = "lastTransferTime=%s" % lastTransferTime
        else:
            postUrl = ""
        return self.make_request("GET", endpoint, postUrl=postUrl)

    # returns all notifications
    def get_notifications(self):
        endpoint = "/api/v3/notifications"
        return self.make_request("GET", endpoint)

    # makes an internal transfer
    def transfer(self, fromAccount, toAccount, unit, amount):
        endpoint = "/api/v3/transfer"
        postBody = "fromAccount=%s&toAccount=%s&unit=%s&amount=%s" % (fromAccount, toAccount, unit, amount)
        return self.make_request("POST", endpoint, postBody=postBody)

    # signs a message
    def sign_message(self, endpoint, postData, nonce=""):
        # step 1: concatenate postData, nonce + endpoint                
        message = postData + nonce + endpoint

        # step 2: hash the result of step 1 with SHA256
        sha256_hash = hashlib.sha256()
        sha256_hash.update(message.encode('utf8'))
        hash_digest = sha256_hash.digest()

        # step 3: base64 decode apiPrivateKey
        secretDecoded = base64.b64decode(self.apiPrivateKey)

        # step 4: use result of step 3 to has the result of step 2 with HMAC-SHA512
        hmac_digest = hmac.new(secretDecoded, hash_digest, hashlib.sha512).digest()

        # step 5: base64 encode the result of step 4 and return
        return base64.b64encode(hmac_digest)

    # creates a unique nonce
    def get_nonce(self):
        # https://en.wikipedia.org/wiki/Modulo_operation
        self.nonce = (self.nonce + 1) & 8191
        return str(int(time.time() * 1000)) + str(self.nonce).zfill(4)

    # sends an HTTP request
    def make_request(self, requestType, endpoint, postUrl="", postBody=""):
        # create authentication headers
        postData = postUrl + postBody

        if self.useNonce:
            nonce = self.get_nonce()
            signature = self.sign_message(endpoint, postData, nonce=nonce)
            authentHeaders = {"APIKey": self.apiPublicKey, "Nonce": nonce, "Authent": signature}
        else:
            signature = self.sign_message(endpoint, postData)
            authentHeaders = {"APIKey": self.apiPublicKey, "Authent": signature}

        # create request
        url = self.apiPath + endpoint + "?" + postUrl
        request = urllib2.Request(url, str.encode(postBody), authentHeaders)
        request.get_method = lambda: requestType

        # read response
        if self.checkCertificate:
            response = urllib2.urlopen(request, timeout=self.timeout)
        else:
            ctx = ssl.create_default_context()
            ctx.check_hostname = False
            ctx.verify_mode = ssl.CERT_NONE
            response = urllib2.urlopen(request, context=ctx, timeout=self.timeout)

        response = response.read().decode("utf-8")

        # return
        return response
    
    def get_bid(self,ticker):
        orderBook=ast.literal_eval(self.get_orderbook(ticker))
        bids=orderBook["orderBook"]["bids"]
        bid=bids[0][0]
        return bid
    
    def get_ask(self,ticker):
        orderBook=ast.literal_eval(self.get_orderbook(ticker))
        asks=orderBook["orderBook"]["asks"]
        ask=asks[0][0]
        return ask
    
    def get_mid(self,ticker):
        bid = self.get_bid(ticker)
        ask = self.get_ask(ticker)
        mid=(bid+ask)/2
        return mid
    
    def get_ticker_info(self, ticker, info):
        all_tickers_info = pd.DataFrame(ast.literal_eval(self.get_instruments().replace("true","True").replace("false","False"))['instruments'])
        ticker_info = all_tickers_info[all_tickers_info["symbol"]==ticker]
        info = ticker_info[info].values[0]
        return info
    
    def get_time_to_expiry(self,ticker):
        time_to_expiry = abs(pd.datetime.today().day - pd.datetime.strptime(self.get_ticker_info(ticker,"lastTradingTime"),"%Y-%m-%dT%H:%M:%S.%fZ").day)/365
        return time_to_expiry 
    
    def get_ticker(self,future_type,expiry):
        """
        Only 2 types of future:
            - monthly M
            - quaterly Q
        """
        all_tickers_info = pd.DataFrame(ast.literal_eval(self.get_instruments().replace("true","True").replace("false","False"))['instruments'])
        futures = all_tickers_info[all_tickers_info['symbol'].str.contains(future_type)]
        futures['lastTradingTime'] = futures['lastTradingTime'].apply(lambda x: pd.datetime.strptime(x,"%Y-%m-%dT%H:%M:%S.%fZ"))
        if expiry=="M":
            result = futures[futures['lastTradingTime']==min(futures['lastTradingTime'])]['symbol'].values[0]
        elif expiry=="Q":
            result = futures[futures['lastTradingTime']==max(futures['lastTradingTime'])]['symbol'].values[0]
        else:
            raise ValueError('Bad expiry type: "' +expiry+'". Must be "Q" or "M"')
        return result
    
    def get_cash(self, currency):
        accountInfo = ast.literal_eval(self.get_accounts())['accounts']
        return accountInfo['cash']['balances'][currency]

In [14]:
spot_public_key = ""
spot_private_key = ""
apis = API_spot(apiPublicKey=spot_public_key, apiPrivateKey=spot_private_key)

In [12]:
fut_public_key = ""
fut_private_key = ""
apif = API_future(apiPublicKey=fut_public_key, apiPrivateKey=fut_private_key)

In [79]:
def get_best_opportunity_ticker_arb_future_spot():
    best_performing_sym=""
    max_perf=-1
    tickers = get_all_future_tickers()
    for ticker in tickers:
        try:
            perf = compute_spot_fut_arb_performance(ticker, 1e3,debug=False)
            if perf>max_perf:
                max_perf=perf
                best_performing_sym = ticker
        except:
            pass
    print("Best opportunity for spot/future arb with "+best_performing_sym + " with "+str(round(100*max_perf,2))+"% return")
    return best_performing_sym

def get_all_future_tickers():
    tickers = []
    for instrument in json.loads(apif.get_instruments())["instruments"]:
        sym = instrument["symbol"]
        if len(sym.split("_"))==3:
            tickers.append(sym.split("_")[1])
    return list(set(tickers))

def get_future_expiry(ticker_future):
    future_expiry_info = ticker_future.split("_")[2]
    y = 2000 + int(future_expiry_info[:2])
    m = int(future_expiry_info[2:4].replace("0",""))
    d = int(future_expiry_info[4:].replace("0",""))
    return datetime.datetime(y,m,d)

def format_ticker_for_spot_api(ticker):
    return "X" + ticker.upper()[:3] + "Z" + ticker.upper()[3:]

def compute_spot_fut_arb_performance(ticker, notional_usd, debug=True):
    """
    ticker: str, ethusd
    """
    ticker_future = apif.get_ticker("fi_"+ticker, "M")
    price_future = apif.get_mid(ticker_future)
    price_spot = apis.get_mid(format_ticker_for_spot_api(ticker))
    # buy btc spot
    notional_crypto = notional_usd/price_spot
    # short btc fut for same notional
    trade_profit_future = notional_crypto*price_future
    profit = trade_profit_future-notional_usd
    days_to_expiry = (get_future_expiry(ticker_future) - datetime.datetime.now()).days
    y = profit/notional_usd
    if debug:
        print("Performance of "+ticker+" spot/future arb strategy:")
        print("    Profit: $", profit )
        print("    Yield: ",round(y*100,2), "%")
        print("    Equivalent annual yield: ",round(365/days_to_expiry*y*100,2), "%")
        print("-----------------------------------------------------------------------")
    return y

def execute_spot_fut_arb(ticker, notional):
    """
    ticker: str, "xbtusd"
    """
    spot_ticker = format_ticker_for_spot_api(ticker)
    future_ticker = apif.get_ticker("fi_"+ticker, "M")
    
    # check we have the notional on kraken spot
    status="not done"
    if float(apis.get_balance("ZUSD"))*1.1>notional:
        # buy the notional in ticker
        apis.create_order(spot_ticker,"buy","market",apis.get_mid("XXBTZUSD"),notional)
        # check order exectuted
        # send the notional to kraken futures
        apis.transfer_from_spot_to_future("XXBT", )
        return status
    else:
        raise ValueError("Not enough USD!")
    
    # short the future
    return status

In [84]:
spot_ticker = format_ticker_for_spot_api(ticker)
future_ticker = apif.get_ticker("fi_"+ticker, "M")

C:\Users\Arthur\Anaconda3\lib\site-packages\ipykernel_launcher.py:504: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy


In [85]:
get_best_opportunity_ticker_arb_future_spot()

C:\Users\Arthur\Anaconda3\lib\site-packages\ipykernel_launcher.py:504: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy


Best opportunity for spot/future arb with ltcusd with 1.18% return


'ltcusd'

In [80]:
apis.get_balance("ZEUR")

'0.0060'

In [86]:
ticker

'xbtusd'

In [92]:
apis.create_order(format_ticker_for_spot_api(ticker),"buy","limit",120,0.001)

{'error': ['EOrder:Insufficient funds']}